In [8]:
%load_ext autoreload
%autoreload 2
%env WANDB_SILENT=true

from pathlib import Path
from copy import deepcopy
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import quantstats as qs
import os
import logging
import sys

qs.extend_pandas()
plt.style.use("seaborn-whitegrid")
pd.set_option('display.max_rows', 200)

if "freqtrade" not in os.listdir():
    import nest_asyncio
    nest_asyncio.apply()
    os.chdir("..")
    
PATH_MOUNT = Path.cwd().parent / "mount"
print("Working Directory:", os.getcwd())

# Freqtrade Imports
from freqtrade.nbtools import remote_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: WANDB_SILENT=true
Working Directory: C:\CS\Python\crypto-collection\freqtrade\superft


## Local Section

In [2]:
# Get local presets
local = (PATH_MOUNT / "presets").glob("*")
local = (it.name for it in list(local) if not it.name.startswith("."))
local_df = pd.DataFrame({"preset_name": local})
local_df

,preset_name
0,15m-next30m-10_06
1,15m-next30m-combine
2,bigz04
3,smaoffset-protect-opt-v1


## Cloud Section

In [4]:
# Preset Backtest Results
df_download = remote_utils.cloud_get_presets_df(from_run_history=False)
df_download

,preset_name,backtest_date,leverage,direction,is_hedging,num_pairs,data_source,win_rate,avg_profit_winners_abs,avg_profit_losers_abs,...,max_drawdown_low,max_drawdown_high,csum_min,csum_max,profit_mean_abs,preset_type,fee,random_name,locks,pairlist
0,ma_crossover__backtest-2021-07-12_22-50-00,2021-07-12_22-50-00,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
1,ma_crossover__backtest-2021-07-12_22-50-00,2021-07-12_22-50-00,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
2,ma_crossover__backtest-2021-07-12_22-55-41,2021-07-12_22-55-41,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
3,ma_crossover__backtest-2021-07-12_22-55-41,2021-07-12_22-55-41,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
4,ma_crossover__dry__run-2021-07-12_23-13-05,2021-07-12_22-55-41,1,long,False,1,binance,0.355623,0.175973,-0.162930,...,-14.728091,-0.048229,985.271909,999.951771,-0.042408,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,AwesomeMacd__backtest-2021-08-19_19-04-26,2021-08-19 19:04:26,1,long,False,60,binance,0.418182,1.375660,-0.854862,...,-30.288204,24.443216,969.711796,1025.707664,0.077902,FilePreset,0.001,furious-rei-computer,[],None
215,Obelisk_Ichimoku_ZEMA_v1__backtest-2021-08-19_...,2021-08-19 19:09:49,1,long,False,60,binance,0.597094,0.528385,-0.546057,...,19.423251,47.522821,999.362947,1104.515932,0.138066,FilePreset,0.001,hardworking-thanos-sword,[],None
216,SMAOffset__backtest-2021-08-19_19-10-28,2021-08-19 19:10:28,1,long,False,60,binance,0.660787,0.512540,-0.712145,...,16.998981,50.324964,1000.492721,1071.570776,0.097111,FilePreset,0.001,fearful-shinchan-monitor,[],None
217,Next30M__backtest-2021-08-19_19-19-53,2021-08-19 19:19:53,1,long,False,60,binance,0.693069,0.294056,-0.298256,...,18.479444,22.634156,1000.231746,1034.013949,0.112257,FilePreset,0.001,satisfied-piccolo-sanitizer,[],None


In [14]:
# Get top 20 strategies by drawdown, but backtest_end is not older than 7 days. Drop duplicates by profit factor.
df = df_download.copy().drop_duplicates(subset=["profit_factor"])

df["profit_mean_pct"] = df["profit_mean"] * 100
df["backtest_start"] = pd.to_datetime(df["backtest_start"])
df["backtest_end"] = pd.to_datetime(df["backtest_end"])

columns = ["preset_name", "profit_mean_pct", "profit_factor", "backtest_start" ,"backtest_end"]

newer = pd.to_datetime(datetime.today() - timedelta(days=7))
last_month_btstart = pd.to_datetime(datetime.today() - timedelta(days=63))

df.loc[
    (df["profit_mean_pct"] > 0.25) & (df["backtest_end"] > newer) & (df["backtest_start"] > last_month_btstart), 
columns].sort_values(by=["profit_factor"], ascending=False).head(20)

,preset_name,profit_mean_pct,profit_factor,backtest_start,backtest_end


In [15]:
df_download.columns

Index(['preset_name', 'backtest_date', 'leverage', 'direction', 'is_hedging',
       'num_pairs', 'data_source', 'win_rate', 'avg_profit_winners_abs',
       'avg_profit_losers_abs', 'sum_profit_winners_abs',
       'sum_profit_losers_abs', 'profit_factor', 'profit_per_drawdown',
       'expectancy_abs', 'total_trades', 'total_volume', 'avg_stake_amount',
       'profit_mean', 'profit_median', 'profit_total', 'profit_total_abs',
       'backtest_start', 'backtest_start_ts', 'backtest_end',
       'backtest_end_ts', 'backtest_days', 'backtest_run_start_ts',
       'backtest_run_end_ts', 'trades_per_day', 'market_change',
       'stake_amount', 'stake_currency', 'stake_currency_decimals',
       'starting_balance', 'dry_run_wallet', 'final_balance',
       'rejected_signals', 'max_open_trades', 'max_open_trades_setting',
       'timeframe', 'timerange', 'enable_protections', 'strategy_name',
       'stoploss', 'trailing_stop', 'trailing_stop_positive',
       'trailing_stop_positive_offs

In [8]:
# Preset Run History
columns = ["preset_name", "profit_mean_pct", "profit_factor"]
df = remote_utils.cloud_get_presets_df(from_run_history=True)
df["profit_mean_pct"] = df["profit_mean"] * 100
df.loc[df["profit_mean_pct"] > 0.25, columns].sort_values(by=["profit_factor"], ascending=False).head(20)

,preset_name,profit_mean_pct,profit_factor
